In [1]:
from database import *

In [ ]:
db = LoadDatabase()

In [36]:
with open("users.pickle", "rb") as infile:
    users = pickle.load(infile)
with open("contests.pickle", "rb") as infile:
    contests = pickle.load(infile)
with open("history.pickle", "rb") as infile:
    history = pickle.load(infile)
with bz2.BZ2File("standings.pickle.bz2", "r") as infile:
    standings = pickle.load(infile)

In [37]:
db = Database(users, contests, history, standings, clean=False)

In [17]:
s = set()
for handle, hist in db.history.items():
    for contestId in hist.contestId:
        if not contestId in db.contests.index:
            s.add(handle)
            break
#             print(contestId, handle)

In [11]:
db = Database(users, contests, history, standings, clean=True)

AssertionError: 